# Análisis de sentimiento

 
## Aprendizaje supervisado: un problema de clasificación

Los algoritmos de aprendizaje supervisado utilizan datos etiquetados en los que tanto la entrada como el resultado objetivo (*etiqueta*), se proporcionan al algoritmo. El aprendizaje supervisado también se denomina modelado predictivo o análisis predictivo, porque crea un modelo que es capaz de realizar predicciones.

Un algoritmo de clasificación toma un conjunto de datos con etiquetas conocidas y características predeterminadas, y aprende cómo etiquetar nuevos registros en función de esa información. Las características definen a cada individuo (cada registro, fila de nuestros datos, también llamado *ejemplo*). La etiqueta es la salida que corresponde con esas características. 

Veamos un ejemplo de clasificación de texto. 
* ¿Qué estamos tratando de predecir?
  * Si una revisión de producto es positiva o negativa.
  * Retrasada es la etiqueta: 1 para positivo 0 para negativo
* ¿Cuáles son las propiedades que puede utilizar para hacer predicciones?
  * Las palabras del texto de revisión se utilizan como características para descubrir similitudes y categorizar el sentimiento del texto del cliente como positivo o negativo.

### Regresión logística

La regresión logística es un método popular para predecir una respuesta binaria. Es un caso especial de modelos lineales generalizados que predice la probabilidad de que la clase asociada a un ejemplo sea una de las clases, o bien la otra (suele usarse casi siempre en problemas donde los registros pertenecen a una de entre dos clases posibles). La regresión logística mide la relación entre la "etiqueta" ***Y*** y las "características" ***X*** a través la estimación de probabilidades mediante una función logística. El modelo predice una probabilidad que se utiliza para predecir la clase a la que pertenece ese ejemplo.

### Dataset de opiniones de Amazon

Se puede descargar desde <a target="_blank" href="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Sports_and_Outdoors_5.json.gz">aquí</a>

In [ ]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Sports_and_Outdoors_5.json.gz
!gunzip reviews_Sports_and_Outdoors_5.json.gz
!hdfs dfs -copyFromLocal reviews_Sports_and_Outdoors_5.json gs://<nombrebucket>/datos

Tenemos un conjunto de datos formado por textos breves escritos por clientes de Amazon al recibir su compra, en los cuales cada cliente expresa su opinión sobre el producto adquirido. Cada registro (cada fila del dataset) representa la opinión frente a algún producto. El texto tiene, por un lado, una columna en la que el cliente da un titular o resumen a su revisión, y por otro, una columna con un texto más largo donde expresa el detalle.

El dataset se encuentra en formato JSON-line en el que cada línea es un JSON completo, como el del siguiente ejemplo:

`{"reviewerID": "A1PUWI9RTQV19S", "asin": "B003Y5C132", "reviewerName": "kris", "helpful": [0, 1], "reviewText": "A little small in hind sight, but I did order a .30 cal box. Good condition, and keeps my ammo organized.", "overall": 5.0, "summary": "Nice ammo can", "unixReviewTime": 1384905600, "reviewTime": "11 20, 2013"}`

que, como vemos, sigue el siguiente esquema:

* **reviewerID** - identificador del cliente, p.ej. A2SUAM1J3GNN3B
* **asin** - identificador del producto, p.ej. 0000013714
* **reviewerName** - nombre del cliente
* **helpful** - valoración del grado de utilidad de esta opinión, expresado como un número real entre 0 y 1, p.ej. 2/3
* **reviewText** - texto de la opinión
* **overall** - valoración que da el cliente al producto, entre 1 y 5
* **summary** - resumen de la revisión
* **unixReviewTime** - instante en el que se creó esta opinión (expresado como unix time)
* **reviewTime** - instante en el que se creó esta opinión (formato en crudo)

In [3]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [4]:
rawDF = spark.read\
           .option("inferSchema", "true")\
           .json("gs://ucmbucket2023/datos/reviews_Sports_and_Outdoors_5.json")

# add column combining summary and review text, drop some others 
df = rawDF.withColumn("reviewTS",
                      F.concat(F.col("summary"), F.lit(" "), F.col("reviewText")))\
          .drop("helpful", "reviewerID", "reviewerName", "reviewTime")

df.show(5)

+----------+-------+--------------------+--------------------+--------------+--------------------+
|      asin|overall|          reviewText|             summary|unixReviewTime|            reviewTS|
+----------+-------+--------------------+--------------------+--------------+--------------------+
|1881509818|    5.0|This came in on t...|      Woks very good|    1390694400|Woks very good Th...|
|1881509818|    5.0|I had a factory G...|Works as well as ...|    1328140800|Works as well as ...|
|1881509818|    4.0|If you don't have...|It's a punch, tha...|    1330387200|It's a punch, tha...|
|1881509818|    4.0|This works no bet...|It's a punch with...|    1328400000|It's a punch with...|
|1881509818|    4.0|I purchased this ...|Ok,tool does what...|    1366675200|Ok,tool does what...|
+----------+-------+--------------------+--------------------+--------------+--------------------+
only showing top 5 rows



In [5]:
df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)
 |-- reviewTS: string (nullable = true)



Vamos a quitar las opiniones neutras (con valoración 3 sobre 5) para evitar posibles confusiones. Cualquier opinión con valoración 1 ó 2 será considerada negativa, mientras que una opinión con valoración 4 ó 5 será considerada positiva. Estas dos clases (negativa y positiva) serán los posibles valores de nuestra columna objetivo.

In [6]:
no_neutral_df = df.filter("overall !=3")
no_neutral_df.show()

+----------+-------+--------------------+--------------------+--------------+--------------------+
|      asin|overall|          reviewText|             summary|unixReviewTime|            reviewTS|
+----------+-------+--------------------+--------------------+--------------+--------------------+
|1881509818|    5.0|This came in on t...|      Woks very good|    1390694400|Woks very good Th...|
|1881509818|    5.0|I had a factory G...|Works as well as ...|    1328140800|Works as well as ...|
|1881509818|    4.0|If you don't have...|It's a punch, tha...|    1330387200|It's a punch, tha...|
|1881509818|    4.0|This works no bet...|It's a punch with...|    1328400000|It's a punch with...|
|1881509818|    4.0|I purchased this ...|Ok,tool does what...|    1366675200|Ok,tool does what...|
|1881509818|    5.0|Needed this tool ...|Glock punch tool ...|    1351814400|Glock punch tool ...|
|1881509818|    5.0|If u don't have i...|          Great tool|    1402358400|Great tool If u d...|
|209486924

La función `describe()` nos da estadísticas de resumen acerca de una o varias columnas numéricas.

In [7]:
no_neutral_df.describe("overall").show()

+-------+------------------+
|summary|           overall|
+-------+------------------+
|  count|            272266|
|   mean|  4.51664548639933|
| stddev|0.9344777791100664|
|    min|               1.0|
|    max|               5.0|
+-------+------------------+



In [8]:
no_neutral_df.groupBy("overall").count().show()

+-------+------+
|overall| count|
+-------+------+
|    1.0|  9045|
|    4.0| 64809|
|    2.0| 10204|
|    5.0|188208|
+-------+------+



## Conversión de la valoración numérica en una etiqueta binaria

Vamos a crear, a partir de la columna `overall` que contiene la valoración numérica, una nueva columna binaria llamada `label` que será la que utilice nuestros algoritmo predictivo. Para ello utilizaremos un `Binarizer` de Spark, fijando el umbral en 3.0 (que nunca se da en nuestros datos porque ya lo habíamos quitado). Todo valor por debajo de este umbral será considerado como 0.0 y todo valor por encima será convertido en 1.0. La columna original `overall` no se modifica.

In [9]:
from pyspark.ml.feature import Binarizer
import numpy as np

binarizer = Binarizer(inputCol = "overall",
                       outputCol = "label",
                        threshold = 3.0)

binary_target_df = binarizer.transform(no_neutral_df)
binary_target_df.groupBy("overall","label").count().show()

+-------+-----+------+
|overall|label| count|
+-------+-----+------+
|    2.0|  0.0| 10204|
|    5.0|  1.0|188208|
|    1.0|  0.0|  9045|
|    4.0|  1.0| 64809|
+-------+-----+------+



## Muestreo estratificado
Como suele ser habitual en los problemas de clasificación binaria, existen muchos más ejemplos pertenecientes a una clase (en este caso la clase positiva) que a otra. Para que el modelo también sea sensible a ejemplos de la clase negativa, es conveniente tratar de equilibrar la proporción de ejemplos de cada clase presentes en nuestro conjunto de datos. Hay varias estrategias para conseguir esto. Aquí optamos por la más simple (y a la vez, la menos recomendable en problemas reales) que es eliminar ejemplos de la clase mayoritaria.

Utilizamos la función `sampleBy()` indicando la fracción de ejemplos de cada clase que queremos mantener. En este caso queremos mantener todos los ejemplos de la clase negativa (que son minoría), pero tan sólo queremos mantener el 10 % de los ejemplos de la clase mayoritaria. Si mostramos la cantidad de ejemplos en el DataFrame resultante de este muestreo, vemos que están más equilibrados aunque aún sigue ligeramente inclinado hacia la clase 1.0.

In [10]:
fractions = {1.0 : .1, 0.0 : 1.0}
balanced_df = binary_target_df.stat.sampleBy("label", fractions, 36)
balanced_df.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|19249|
|  1.0|25269|
+-----+-----+



Para poder saber cómo de bien funcionará el modelo entrenado en datos nuevos nunca vistos, vamos a dividir el conjunto de datos en subconjuntos de entrenamiento y de test. El conjunto de test se utilizará una sola vez, al final, cuando ya tengamos decidido y entrenado el modelo de predicción. El objetivo del conjunto de test será calcular una métrica que estime la bondad del modelo cuando sea puesto en producción y empiece a predecir datos sobre los que realmente no se conoce su etiqueta.

Usamos el 80 % de nuestros datos para entrenar, y el 20 % los dejamos fuera porque serán el conjunto de test.

In [11]:
split_seed = 5043
training_data, test_data = balanced_df.randomSplit([0.8, 0.2], split_seed)

training_data.cache()
training_data.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|15416|
|  1.0|20214|
+-----+-----+



## Ingeniería de variables y pipelines

Para que las características sean utilizadas por un algoritmo de aprendizaje automático, deben transformarse y colocarse en vectores de características, que son vectores numéricos que representa el valor de cada característica. Los textos en sí mismos no son utilizables por los algoritmos hasta que no pasen a través de dicho proceso.

Spark ML proporciona un conjunto uniforme de API de alto nivel creadas sobre DataFrames. Usaremos un ML Pipeline para pasar los datos a través de transformadores con el fin de extraer las características y un estimador para producir el modelo.

* Transformador: Un transformador es un algoritmo que transforma un DataFrame en otro DataFrame. Usaremos un transformador para obtener un DataFrame con una columna de vector de características.

* Estimador: un estimador es un algoritmo que se puede ajustar a un DataFrame para producir un transformador. Usaremos un estimador que consistirá en un algoritmo de Regresión logística para entrenar un modelo. El modelo entrenado obtenido será un transformador que es capaz de transformar datos sobre los que no se conoce su etiqueta, para calcular predicciones.

* Pipeline: un pipeline encadena varios transformadores y estimadores para especificar un flujo de trabajo de aprendizaje automático. Usaremos un Pipeline de Spark ML para tener en una sola pieza toda la secuencia de transformaciones necesarias para preparar los datos hasta llegar al modelo. De esa manera, podemos entrenar la pieza (el pipeline) como un todo, y utilizarlo para que los datos nuevos también pasen a través de las mismas etapas que habíamos utilizado para preparar los datos de entrenamiento. Esto permite pre-procesar datos nuevos de la misma manera que se hizo con los datos de entrenamiento, siguiendo exactamente los mismos pasos.

Por último utilizaremos un evaluador para medir la bondad del modelo entrenado.

Empezaremos con las siguientes etapas de ingeniería de variables:

* Primero utilizamos un `RegexTokenizer` para separar cada texto en palabras. Esto transforma cada texto en un vector de strings con las palabras. Para más detalles: http://spark.apache.org/docs/latest/ml-features.html#tokenizer
* Después aplicaremos un `StopWordsRemover` para eliminar de cada vector de palabras aquellas sin significado, como artículos, preposiciones, etc. Para más detalles: http://spark.apache.org/docs/latest/ml-features.html#stopwordsremover


In [12]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover

tokenizer = RegexTokenizer(inputCol = "reviewTS",
                               outputCol = "reviewTokensUf",
                               pattern = "\\s+|[,.()\"]")

remover = StopWordsRemover(inputCol = "reviewTokensUf",
                           outputCol = "reviewTokens"
                          ).setStopWords(StopWordsRemover.loadDefaultStopWords("english"))

Utilizaremos el siguiente método para convertir vectores de palabras de un texto en vectores numéricos, utilizables por un algoritmo predictivo.

**De la documentación oficial de Spark**:

TF-IDF es un método de vectorización de características ampliamente utilizado en la minería de texto para reflejar la importancia de un término para un documento en el corpus. Si denotamos a un término (palabra) como *t*, un documento como *d* y el corpus como *D*, entonces:

* La Frecuencia de un término **TF(*t*, *d*)** es el número de veces que el término *t* aparece en el documento *d* 
* La frecuencia en el documento **DF(*t*, *D*)** es el número de documentos que contienen el término *t*.

Si solo usamos la frecuencia de los términos para medir la importancia, es muy fácil sobreenfatizar erróneamente los términos que aparecen con mucha frecuencia pero que contienen poca información sobre el documento, p. Ej. la palabra *fútbol* en un corpus compuesto por biografías de futbolistas. Si un término aparece con mucha frecuencia en el corpus, significa que no contiene información especial sobre un documento en particular. 

* La *frecuencia inversa de los documentos* **IDF(*t*, *D*)** es una medida numérica de cuánta información proporciona un término:

$$ IDF (t, D) = \frac{log | D | +1} {DF (t, D) +1} $$

donde | D | es el número total de documentos del corpus. Dado que se usa logaritmo, si un término aparece en todos los documentos, su valor IDF se convierte en 0. Tenga en cuenta que se aplica un término de suavizado para evitar dividir por cero para los términos fuera del corpus.

La medida TF-IDF es simplemente el producto de TF e IDF:
$$ TFIDF (t, d, D) = TF (t, d) ⋅ IDF (t, D) $$

Hay varias variantes en la definición de TF y de IDF. En Spark ML, están separados para que sean flexibles y poder combinarlos de varias maneras.

* `CountVectorizer()` cuenta las ocurrencias totales de cada palabra en todo el corpus de textos, y se queda con las N más relevantes, siendo N un parámetro especificado por el usuario (en nuestro caso, N = 20000). Tras esto, en cada texto contará el número de apariciones de cada una de esas N palabras seleccionadas. Por tanto, cada texto vendrá representado por un vector numérico de longitud 20000, y nuestro problema tendrá 20000 variables.

* `HashingTF()` es similar, pero cada posición no se asocia a una sola palabra sino que puede estar compartida por más de una. El usuario especifica también la dimensión N de los vectores obtenidos (se recomienda que sea una potencia de 2 debido a cómo actúa esta técnica). A grandes rasgos, cada palabra se codifica mediante un código que a su vez va a parar a una posición determinada del vector numérico que va a representar a ese texto, por lo que puede haber colisiones en algunas ocasiones, y que una posición sea utilizada para acumular las apariciones de más de una palabra diferente.

Se puede utilizar cualquiera de estas opciones, aunque no las dos simultáneamente.

In [13]:
from pyspark.ml.feature import CountVectorizer, IDF, HashingTF

count_vectorizer = CountVectorizer().setInputCol("reviewTokens")\
                                    .setOutputCol("cv")\
                                    .setVocabSize(20000).setMinDF(4)

idf = IDF().setInputCol("cv").setOutputCol("features")

In [14]:
from pyspark.ml.classification import LogisticRegression

# El último elemento del pipeline será un estimador, concretamente de la clase LogisticRegression
logisticRegression = LogisticRegression().setMaxIter(100)\
                                 .setRegParam(0.02)\
                                 .setElasticNetParam(0.3)

## Configuramos el pipeline

In [16]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages = [tokenizer, remover, count_vectorizer, idf, logisticRegression])

### Entrenamos el pipeline, como un todo

Recordemos que ahora mismo está activada la etapa `CountVectorizer`, con lo que ignoramos completamente el `HashingTF`. Nos servirá en el futuro cuando queramos decidir cuál de las dos opciones funciona mejor en base a su resultado.

In [17]:
pipelineModel = pipeline.fit(training_data)

23/03/18 08:57:30 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/03/18 08:57:30 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [18]:
import pandas as pd

vocabulary = pipelineModel.stages[2].vocabulary

# De la lista de stages, nos quedamos con el último elemento (LogisticRegressionModel, ya entrenado: transformador)
lrModel = pipelineModel.stages[-1]

# Obtenemos el array de coeficientes, que vienen en el mismo orden que las variables
weights = lrModel.coefficients

# Lista de pares (palabra, coeficiente) 
word_weight = list(zip(vocabulary,weights))

word_weight.sort(key = lambda pair: np.abs(pair[1]), reverse = True)

# Convertimos la lista de pares en un DataFrame para poder imprimirlo de manera más clara
word_weight_df = pd.DataFrame(word_weight, columns = ["word", "weight"])[0:20]
word_weight_df

,word,weight
0,great,0.592809
1,returned,-0.377652
2,poor,-0.326958
3,perfect,0.320517
4,useless,-0.298490
5,waste,-0.286789
6,broke,-0.270305
7,easy,0.263212
8,junk,-0.255344
9,return,-0.254468


# Predicciones en tiempo real con el modelo entrenado

Vamos a usar Spark Structured Streaming para hacer predicciones. Aunque los datos originales tenían dos columnas separadas `summary` y `reviewText`, desde el principio habíamos concatenado ambas en una sola columna `reviewTS` que es la que se utiliza como punto de partida en el pipeline, que no necesita para nada `reviewText` ni `summary`. Por tanto nuestros datos en streaming tendrán una sola columna de tipo cadena de caracteres (string) llamada `reviewTS` que contiene en cada fila un texto completo (un string muy largo). 

Tampoco necesitamos ninguna columna de `label` ni valoración ni similar, puesto que son datos para predecir y asumimos que no tenemos por qué conocer dichos atributos.

Vamos a dar cada dato como si fuera un JSON completo en una sola línea. Después despiezamos cada JSON (cada línea) para pasarlo a un DataFrame de una columna de tipo string. Leermos cada JSON como una única línea de tipo string obtenida de Apache Kafka, configurando las siguientes opciones:

  * Usamos la variable `readStream` (en lugar de `read` como solemos hacer) interna de la SparkSession `spark`
  * Indicamos que el formato es `"kafka"` con `.format("kafka")`
  * Indicamos cuáles son los brokers de Kafka de los que vamos a leer y el puerto al que queremos conectarnos para leer (9092 es el que usa Kafka por defecto), con `.option("kafka.bootstrap.servers", "<nombre_cluster>-w-0:9092,<nombre_cluster>-w-1:9092")`. De esa manera podremos leer el mensaje si el productor de Kafka lo envía a cualquiera de los dos brokers existentes, que son los nodos del cluster identificados como `<nombre_cluster>-w-0` y `<nombre_cluster>-w-1`
  * Indicamos que queremos subscribirnos al topic `"revisiones"` con `.option("subscribe", "revisiones")`.
  * Finalmente ponemos `load()` para realizar la lectura.

#### Creamos (desde línea de comandos) el topic revisiones en Kafka
`/usr/lib/kafka/bin/kafka-topics.sh --zookeeper localhost:2181 --create --replication-factor 1 --partitions 1 --topic revisiones`

#### Vemos los topics existentes
`/usr/lib/kafka/bin/kafka-topics.sh --zookeeper localhost:2181 --list`

#### Abrimos el Kafka console producer (productor de Kafka desde consola, para enviarle mensajes al broker 0 del cluster de Kakfa)
`/usr/lib/kafka/bin/kafka-console-producer.sh --broker-list <nombrecluster>-w-0:9092 --topic revisiones`

#### Escribimos como mensajes: 
```
{"reviewTS": "This is an absolutely horrible product, what a shit!"}
{"reviewTS": "The best purchase I have ever done, awesome"}
```

In [19]:
# Leemos de Kafka suscribiéndonos al topic "revisiones" (revisiones de productos)
textosStreamingDF = spark.readStream\
  .format("kafka")\
  .option("kafka.bootstrap.servers", "ucmcluster-w-0:9092,ucmcluster-w-1:9092")\
  .option("subscribe", "revisiones")\
  .load()

In [20]:
textosStreamingDF.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [21]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql import functions as F

# Este es el esquema de cada JSON: un único campo. 
# Spark lo parsea como una columna de tipo estructura que dentro tiene un único campo
esquema = StructType([\
  StructField("reviewTS", StringType())\
])

parsedDF = textosStreamingDF\
    .withColumn("value", F.col("value").cast(StringType()))\
    .withColumn("reviewStruct", F.from_json(F.col("value"), esquema))\
    .withColumn("reviewTS", F.col("reviewStruct.reviewTS"))

predictionsStreamingDF = pipelineModel.transform(parsedDF)

In [22]:
parsedDF.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: string (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)
 |-- reviewStruct: struct (nullable = true)
 |    |-- reviewTS: string (nullable = true)
 |-- reviewTS: string (nullable = true)



In [23]:
predictionsStreamingDF.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: string (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)
 |-- reviewStruct: struct (nullable = true)
 |    |-- reviewTS: string (nullable = true)
 |-- reviewTS: string (nullable = true)
 |-- reviewTokensUf: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- reviewTokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- cv: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [24]:
output = predictionsStreamingDF\
                    .writeStream\
                    .queryName("predicciones")\
                    .outputMode("append")\
                    .format("memory")\
                    .start()

23/03/18 10:23:33 WARN org.apache.spark.sql.streaming.StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-4b20346a-c1ef-4cab-8074-312d82a50085. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/03/18 10:23:33 WARN org.apache.spark.sql.streaming.StreamingQueryManager: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [26]:
prediccionesDF = spark.sql("select * from predicciones")
prediccionesDF.show(truncate = False)

+----+--------------------------------------------------------------------+----------+---------+------+-----------------------+-------------+------------------------------------------------------+----------------------------------------------------+-------------------------------------------------------------+--------------------------------------+----------------------------------+-----------------------------------------------------------------------------+--------------------------------------+---------------------------------------+----------+
|key |value                                                               |topic     |partition|offset|timestamp              |timestampType|reviewStruct                                          |reviewTS                                            |reviewTokensUf                                               |reviewTokens                          |cv                                |features                                                     

In [27]:
prediccionesDF.show(truncate = False)

+----+--------------------------------------------------------------------+----------+---------+------+-----------------------+-------------+------------------------------------------------------+----------------------------------------------------+-------------------------------------------------------------+--------------------------------------+--------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------+----------------------------------------+---------------------------------------+----------+
|key |value                                                               |topic     |partition|offset|timestamp              |timestampType|reviewStruct                                          |reviewTS                                            |reviewTokensUf                                               |reviewTokens                          |cv                                